<a href="https://colab.research.google.com/github/Wiickz/MAT421/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Section 19.1: Root Finding Problem Statement

The goal of this chapter is to explore various methods for finding roots/zeros of a function. While the roots of some functions can easily be determined algebraically, others may be difficult to compute by hand:

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import optimize

f = lambda x: np.cos(x) - x # example function - algebra does not help us well here
x = np.linspace(-2, 2, 100)

r = optimize.fsolve(f, 0.5) # finds the root of f near 0.5
print(r)

plt.plot(x, f(x), 'b')
plt.plot(x, np.zeros_like(x), 'r--')
plt.plot(r, 0, 'ro')
plt.text(r[0], 0, '(%.4f, 0)' % r[0], verticalalignment='bottom')
plt.grid()
plt.show()

## Section 19.2: Tolerance

Root-finding programs often do so via numerical computations that converge towards the nearest zero over some number of iterations. Since this iterative method only approaches (and never truly reaches) the zero point, these functions usually have a built-in tolerance level at which the iteration stops and the approximate zero is returned:

In [ ]:
#Code until end of fixedpt() function is excerpt of own work from MAT 423 - Numerical Analysis I
#Fixed point iteration method to find a root of a function

import numpy as np

def fixedpt(func, x0, k=50, tol=1e-6):
    '''Fixed point iteration'''

    x = x0
    x_list = [x]

    for i in range(k):
        x = func(x)
        x_list.append(x)

        if abs(x - x_list[-2]) < tol:
            n = i
            break

    return x_list, n

#Fixed point iteration uses the original function f(x) to create a new function g(x) = x - f(x), which is then used to find the root of f(x)

f_x = lambda x: 0.5 - x + 0.2*np.sin(x) # assumed to be = 0
g_x = lambda x: 0.5 + 0.2*np.sin(x) # g(x) = x - f(x) = 0 -> x = g(x)

x0 = 0.5 # initial guess
tol = 1e-6 # tolerance - try changing this to see how it affects the number of iterations

x_list, n = fixedpt(g_x, x0, tol=tol)

print('Root:', x_list[-1])
print('Number of iterations:', n)
print('Convergence:')
for i, x in enumerate(x_list):
    print('x_%d = %.7f' % (i, x))

The solution found above is accurate to the sixth decimal place, and the convergence list shows how the solution approached this tolerance. For most purposes, six digits of accuracy is more than enough, which is why this particular tolerance was chosen - anything more would require more iterations (and thus computation time) for a likely unneeded level of accuracy.

## Section 19.3: Bisection Method

The bisection method works much in the same way as a binary search - given a function and two values of x, the algorithm assumes that the function is continuous, that one of the x values returns a value less than zero when input to the function, and that the other returns a value greater than zero. With these assumptions, the Intermediate Value Theorem kicks into play - somewhere between these two values, another value *must* exist where f(x) = 0. The algorithm now looks at the midpoint, checks whether it returns a value greater or less than zero, then sets the midpoint as one of the new bounds depending on what it finds. This algorithm repeats until the specified tolerance is reached:

In [ ]:
import numpy as np

# Yes, I also did this in 423. The function here is modified from that work, however -
# for some reason, my 423 work assumed it would always converge.
# I've added a maximum number of iterations for this version.

def bisection(func, a, b, k=50, tol=1e-6):
    '''Bisection algorithm'''
    m = (a + b) / 2
    for i in range(k):
        if abs(func(m)) < tol:
            n = i
            break

        if func(a) * func(m) <= 0:
            b = m
        else:
            a = m

        m = (a + b) / 2

    return m, n


f_x = lambda x: x**2 - x**3 + (x+1)**2 - 2 # function

# guessing interval
a = -3
b = 2
# note there are multiple zeros within this interval - more on this later

tol = 1e-6

root, n = bisection(f_x, a, b, tol=tol)

print('Root:', root)
print('Number of iterations:', n)

It is also worth noting that this method has a form of bias. Depending on the implementation, when multiple zeros exist on the interval, the algorithm will always converge to a particular zero. In the above implementation, the algorithm will always return a zero to the left of the midpoint - usually the leftmost zero, but this is not always the case.

## Section 19.4: Newton-Raphson Method

This method for finding roots does away with the Intermediate Value Theorem and boundary guesses, making use of a much simpler method where the algorithm assumes calculus has been invented and instead uses the input function, its derivative and a single guess. The Newton-Raphson method is still iterative, refining its guess over time by plugging that guess into both the function and its derivative:

In [ ]:
import numpy as np

def newton(func, dfunc, x0, k=50, tol=1e-6):
    '''Newton's method'''
    x = x0

    for i in range(k):
        x = x - func(x) / dfunc(x)

        if abs(func(x)) < tol:
            n = i
            break

    return x, n

f_x = lambda x: 4*np.sin(x) - np.exp(x)
d_f_x = lambda x: 4*np.cos(x) - np.exp(x)

x0 = 0.5 # guess

root, n = newton(f_x, d_f_x, x0)

print('Root:', root)
print('Number of iterations:', n)

Since this algorithm uses a single guess instead of a boundary, it will typically simply converge to the nearest zero rather than work under a specific bias. Regardless of the zero it does find, it usually does so in significantly fewer iterations.